In [18]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
import os

# Replace with your Azure Blob Storage account URL
storage_account_url = "https://msrchatsa.blob.core.windows.net"

# Use DefaultAzureCredential to authenticate using the logged-in user's account
credential = DefaultAzureCredential()

# Create a BlobServiceClient using the storage account URL and the credential
blob_service_client = BlobServiceClient(account_url=storage_account_url, credential=credential)

blob_client = blob_service_client.get_blob_client(container="test-blob", blob="test-blob.txt")

for filename in os.listdir('json'):
    if filename.endswith('.json'):
        with open(f'json/{filename}', 'rb') as data:
            blob_client.upload_blob(data, overwrite=True)

In [4]:
import json
import os
from bs4 import BeautifulSoup
import re
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError

# read in JSON files

data_to_import = []

# Replace with your Azure Blob Storage account URL
storage_account_url = "https://msrchatsa.blob.core.windows.net"

# Use DefaultAzureCredential to authenticate using the logged-in user's account
credential = DefaultAzureCredential()

# Create a BlobServiceClient using the storage account URL and the credential
blob_service_client = BlobServiceClient(account_url=storage_account_url, credential=credential)

def clean_html_content(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'lxml')
    
    # Extract the text content from the parsed HTML
    text_content = soup.get_text(separator=' ')
    
    # Remove extra whitespace and tabs
    clean_text = re.sub(r'\s+', ' ', text_content).strip()
    
    return clean_text

def check_blob_exists(container_name, blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    try:
        blob_client.get_blob_properties()
        return True
    except ResourceNotFoundError:
        return False

# Function to upload a single blob
def upload_blob(container_name, blob_name, content):
    if(check_blob_exists(container_name, blob_name)):
        print(f"Blob {blob_name} already exists in container {container_name}.")
        return
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(content, overwrite=True)
    print(f"Uploaded {blob_name} to container {container_name}.")

blobs_to_upload = []
for filename in os.listdir('json'):
    if filename.endswith('.json'):
        with open(os.path.join('json', filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            for item in data:
                new_item = {
                    'title': item['title'],
                    'content': clean_html_content(item['content']),
                }

                blobs_to_upload.append(new_item)

for item in blobs_to_upload:
    upload_blob('content', f"{item['title']}.txt", item['content'])

Uploaded Research Focus: Week of May 27, 2024.txt to container msrchatssi5prod-chunks.
Uploaded The Crossroads of Innovation and Privacy: Private Synthetic Data for Generative AI.txt to container msrchatssi5prod-chunks.
Uploaded Ideas: Designing AI for people with Abigail Sellen.txt to container msrchatssi5prod-chunks.
Uploaded GigaPath: Whole-Slide Foundation Model for Digital Pathology.txt to container msrchatssi5prod-chunks.
Uploaded Abstracts: May 20, 2024.txt to container msrchatssi5prod-chunks.
Uploaded What’s Your Story: Jacki O’Neill.txt to container msrchatssi5prod-chunks.
Uploaded Research Focus: Week of May 13, 2024.txt to container msrchatssi5prod-chunks.
Uploaded Microsoft at CHI 2024: Innovations in human-centered design.txt to container msrchatssi5prod-chunks.
Uploaded RASCAL: Novel robotics for scalable and highly available automated storage and retrieval.txt to container msrchatssi5prod-chunks.
Uploaded MatterSim: A deep-learning model for materials under real-world co